In [2]:
import numpy as np
import os

from operator import itemgetter
from pathlib import Path

In [5]:
np.random.seed(7)
print(np.random.randint(1, 1001, 10))

# np.random.seed(7)
print(np.random.randint(1, 1001, 20))


[176 197 538 503 580 212 920 616 349 186]
[399 536 585 346 367 555 731 905 168 999  69 433 896 392 941 513  76 824
 251   7]


In [2]:
def generate_knapsack_data(p=3, n=10):    
    data = {'value': [], 'weight': [], 'capacity': 0}    
    
    # Value
    for _ in range(p):
        data['value'].append(np.random.randint(1, 1001, n))    
    # Cost
    data['weight'] = np.random.randint(1, 1001, n)
    # Capacity
    data['capacity'] = np.ceil(0.5 * (np.sum(data['weight'])))
    
    return data

In [8]:
def write_to_file(path, p, n, data):
    text = f'{p}\n{n}\n'
    for i in range(p):        
        string = " ".join([str(v) for v in data['value'][i]])        
        text += string + "\n"
    string = " ".join([str(w) for w in data['weight']])    
    text += string + "\n"
    text += str(int(data['capacity']))

    path.open('w').write(text)    

In [48]:
def get_orderings(data):    
    # TODO add a couple of more
    order = {
        'max_weight': None,
        'min_weight': None,        
        'max_avg_profit': None,
        'min_avg_profit': None,
        'max_max_profit': None,
        'min_max_profit': None,
        'max_min_profit': None,
        'min_min_profit': None,
        'max_avg_profit_by_weight': None,
        'max_max_profit_by_weight': None,
    }
    
    n_items = len(data['weight'])
    for o in order.keys():
        if o == 'max_weight':
            idx_weight = [(i, w) for i, w in enumerate(data['weight'])]
            print(o, idx_weight)
            idx_weight.sort(key=itemgetter(1), reverse=True)
            order[o] = [i[0] for i in idx_weight]
        
        
        elif o == 'min_weight':
            idx_weight = [(i, w) for i, w in enumerate(data['weight'])]
            idx_weight.sort(key=itemgetter(1))
            order[o] = [i[0] for i in idx_weight]
            
            
        elif o == 'max_avg_profit':
            mean_profit = np.mean(data['value'], 0)
            idx_profit = [(i, mp) for i, mp in enumerate(mean_profit)]
            idx_profit.sort(key=itemgetter(1), reverse=True)
            order[o] = [i[0] for i in idx_profit]
            
            
        elif o == 'min_avg_profit':
            mean_profit = np.mean(data['value'], 0)
            idx_profit = [(i, mp) for i, mp in enumerate(mean_profit)]
            idx_profit.sort(key=itemgetter(1))
            order[o] = [i[0] for i in idx_profit]
        
        
        elif o == 'max_max_profit':
            max_profit = np.max(data['value'], 0)
            idx_profit = [(i, mp) for i, mp in enumerate(max_profit)]
            idx_profit.sort(key=itemgetter(1), reverse=True)
            order[o] = [i[0] for i in idx_profit]
            
        elif o == 'min_max_profit':
            max_profit = np.max(data['value'], 0)
            idx_profit = [(i, mp) for i, mp in enumerate(max_profit)]
            idx_profit.sort(key=itemgetter(1))
            order[o] = [i[0] for i in idx_profit]
        
        elif o == 'max_min_profit':
            min_profit = np.min(data['value'], 0)
            idx_profit = [(i, mp) for i, mp in enumerate(min_profit)]
            idx_profit.sort(key=itemgetter(1), reverse=True)
            order[o] = [i[0] for i in idx_profit]
        
        elif o == 'min_min_profit':
            min_profit = np.min(data['value'], 0)
            idx_profit = [(i, mp) for i, mp in enumerate(min_profit)]
            idx_profit.sort(key=itemgetter(1))
            order[o] = [i[0] for i in idx_profit]
            
        elif o == 'max_avg_profit_by_weight':
            mean_profit = np.mean(data['value'], 0)            
            profit_by_weight = [v/w for v, w in zip(mean_profit, data['weight'])]            
            idx_profit_by_weight = [(i, f) for i, f in enumerate(profit_by_weight)]
            idx_profit_by_weight.sort(key=itemgetter(1), reverse=True)
            # print(idx_profit_by_weight)
            order[o] = [i[0] for i in idx_profit_by_weight]
                    
        elif o == 'max_max_profit_by_weight':
            max_profit = np.max(data['value'], 0)     
            print(max_profit)       
            profit_by_weight = [v/w for v, w in zip(max_profit, data['weight'])]
            idx_profit_by_weight = [(i, f) for i, f in enumerate(profit_by_weight)]
            idx_profit_by_weight.sort(key=itemgetter(1), reverse=True)
            print(idx_profit_by_weight)
            order[o] = [i[0] for i in idx_profit_by_weight]
            
                
    return order

In [6]:
obj_var_cfg = {3: [20, 40, 60, 100], 
                5: [20, 40], 
                7: [20, 40]}
   
def generate_dataset(obj_var_cfg, seed=7, n_instances=10000, n_train=8000, n_val=1000, n_test=1000, root='dataset'):
    assert n_instances == n_train + n_val + n_test
    np.random.seed(seed)    
    root = Path(root)
    root = root / 'knapsack' / f'{seed}'
    root.mkdir(parents=True, exist_ok=True)
    
    for p in obj_var_cfg.keys():
        for n in obj_var_cfg[p]:  
            prob_root = root / f'{p}_{n}'
            prob_root.mkdir(parents=True, exist_ok=True)

            dataset = []              
            for i in range(n_instances):            
                data = {'value': [], 'weight': [], 'capacity': 0}    
                # Value
                for _ in range(p):
                    data['value'].append(np.random.randint(1, 1001, n))    
                # Cost
                data['weight'] = np.random.randint(1, 1001, n)
                # Capacity
                data['capacity'] = np.ceil(0.5 * (np.sum(data['weight'])))
                dataset.append(data)
            
            set_root = prob_root / 'train'
            set_root.mkdir(parents=True, exist_ok=True)
            for i in range(n_train): 
                inst_root = set_root / f'kp_{seed}_{p}_{n}_{i}.dat'               
                write_to_file(inst_root, p, n, dataset[i])
                
            set_root = prob_root / 'val'
            set_root.mkdir(parents=True, exist_ok=True)
            for i in range(n_train, n_train + n_val): 
                inst_root = set_root / f'kp_{seed}_{p}_{n}_{i}.dat'
                write_to_file(inst_root, p, n, dataset[i])
            
            set_root = prob_root / 'test'
            set_root.mkdir(parents=True, exist_ok=True)
            for i in range(n_train + n_val, n_train + n_val + n_test): 
                inst_root = set_root / f'kp_{seed}_{p}_{n}_{i}.dat'             
                write_to_file(inst_root, p, n, dataset[i])
            
                # order = get_orderings(data)
                # for o in range(10):
                #     output_filename = f'kp_{p}_{n}_{i}_rand_{o}'
                    
                # for o in order.keys():                
                #     output_filename = f'kp_{p}_{n}_{i}_{o}'
                


In [9]:
generate_dataset(obj_var_cfg=obj_var_cfg)

In [32]:
def read_from_file(p, filepath):
    data = {'value': [], 'weight': [], 'capacity': 0}

    with open(filepath, 'r') as fp:
        fp.readline()
        fp.readline()

        for _ in range(p):
            data['value'].append(list(map(int, fp.readline().split())))
            
        data['weight'].extend(list(map(int, fp.readline().split())))

        data['capacity'] = int(fp.readline().split()[0])
        
    return data

In [33]:
dataset = Path('/home/rahul/Documents/PhD/projects/multiobjective_cp2016/dataset/')
problem_type = 'knapsack'
seed = '7'
n_instances = 500

root = dataset / problem_type / seed
for p in obj_var_cfg.keys():
    for n in obj_var_cfg[p]:
        root_pn = root / f'{p}_{n}'
        for i in range(n_instances):
            filepath = root_pn / 'train' / f'kp_{seed}_{p}_{n}_{i}.dat'            
            data = read_from_file(p, filepath)
            orderings = get_orderings(data)
            for k, v in orderings.items():
                print(k, v)
            
            break
        break
    break
            

{'value': [[176, 197, 538, 503, 580, 212, 920, 616, 349, 186, 399, 536, 585, 346, 367, 555, 731, 905, 168, 999], [69, 433, 896, 392, 941, 513, 76, 824, 251, 7, 788, 445, 45, 704, 326, 825, 153, 184, 950, 113], [764, 190, 961, 291, 313, 202, 463, 551, 773, 495, 138, 856, 484, 196, 73, 852, 945, 258, 705, 401]], 'weight': [375, 928, 350, 941, 605, 765, 584, 280, 523, 746, 804, 449, 318, 392, 152, 93, 760, 225, 662, 357], 'capacity': 5155}
[336.33333333 273.33333333 798.33333333 395.33333333 611.33333333
 309.         486.33333333 663.66666667 457.66666667 229.33333333
 441.66666667 612.33333333 371.33333333 415.33333333 255.33333333
 744.         609.66666667 449.         607.66666667 504.33333333]
[0.8968888888888888, 0.29454022988505746, 2.280952380952381, 0.4201204392490258, 1.0104683195592288, 0.403921568627451, 0.8327625570776256, 2.3702380952380953, 0.8750796685787126, 0.30741733690795353, 0.5493366500829188, 1.3637713437268004, 1.1677148846960168, 1.0595238095238095, 1.67982456140

In [49]:
filepath = Path(
    '/home/rahul/Documents/PhD/projects/multiobjective_cp2016/data/knapsack/set1/knapsack-10-3-1000-6464.dat')

data = read_from_file(3, filepath)
get_orderings(data)


max_weight [(0, 158), (1, 737), (2, 673), (3, 930), (4, 826), (5, 626), (6, 354), (7, 117), (8, 118), (9, 839)]
[474 130 666 490 602 722 939 793 485 749]
[(7, 6.777777777777778), (8, 4.110169491525424), (0, 3.0), (6, 2.652542372881356), (5, 1.1533546325878594), (2, 0.9895988112927192), (9, 0.8927294398092968), (4, 0.7288135593220338), (3, 0.5268817204301075), (1, 0.17639077340569878)]


{'max_weight': [3, 9, 4, 1, 2, 5, 6, 0, 8, 7],
 'min_weight': [7, 8, 0, 6, 5, 2, 1, 4, 9, 3],
 'max_avg_profit': [6, 5, 4, 9, 2, 8, 3, 7, 0, 1],
 'min_avg_profit': [1, 0, 7, 3, 8, 2, 9, 4, 5, 6],
 'max_max_profit': [6, 7, 9, 5, 2, 4, 3, 8, 0, 1],
 'min_max_profit': [1, 0, 8, 3, 4, 2, 5, 9, 7, 6],
 'max_min_profit': [4, 5, 2, 8, 6, 9, 3, 0, 7, 1],
 'min_min_profit': [1, 7, 0, 3, 9, 6, 8, 2, 5, 4],
 'max_avg_profit_by_weight': [8, 7, 6, 0, 5, 2, 4, 9, 3, 1],
 'max_max_profit_by_weight': [7, 8, 0, 6, 5, 2, 9, 4, 3, 1]}

max_weight [(0, 158), (1, 737), (2, 673), (3, 930), (4, 826), (5, 626), (6, 354), (7, 117), (8, 118), (9, 839)]


{'max_weight': [3, 9, 4, 1, 2, 5, 6, 0, 8, 7],
 'min_weight': [7, 8, 0, 6, 5, 2, 1, 4, 9, 3],
 'max_avg_profit': [6, 5, 4, 9, 2, 8, 3, 7, 0, 1],
 'min_avg_profit': [1, 0, 7, 3, 8, 2, 9, 4, 5, 6],
 'max_max_profit': [6, 7, 9, 5, 2, 4, 3, 8, 0, 1],
 'min_max_profit': [1, 0, 8, 3, 4, 2, 5, 9, 7, 6],
 'max_min_profit': [4, 5, 2, 8, 6, 9, 3, 0, 7, 1],
 'min_min_profit': [1, 7, 0, 3, 9, 6, 8, 2, 5, 4],
 'max_avg_profit_by_weight': [8, 7, 6, 0, 5, 2, 4, 9, 3, 1],
 'max_max_profit_by_weight': [7, 8, 0, 6, 5, 2, 9, 4, 3, 1]}

In [8]:
seed = 7
root_path = Path(f'/scratch/rahulpat/knapsack/{seed}')
n_instances = 250

obj_var_cfg = ['3_20', '3_40', '3_60', '3_80', '3_100']
# ['5_20', '7_20', '3_40', '5_40', '7_40', '3_60', '5_60', '3_100']

for ovc in obj_var_cfg:
    for i in range(n_instances):
        f = root_path / ovc / f'train/kp_{seed}_{ovc}_{i}.dat'
        entry = " ".join(["/home/rahulpat/projects/def-khalile2/rahulpat/multiobjbdd/code/multiobj",
                 str(f), "1", "0", f"./output.$SLURM_JOB_ID"])
        print(entry)


/home/rahulpat/projects/def-khalile2/rahulpat/multiobjbdd/code/multiobj /scratch/rahulpat/knapsack/7/3_20/train/kp_7_3_20_0.dat 1 0 ./output.$SLURM_JOB_ID
/home/rahulpat/projects/def-khalile2/rahulpat/multiobjbdd/code/multiobj /scratch/rahulpat/knapsack/7/3_20/train/kp_7_3_20_1.dat 1 0 ./output.$SLURM_JOB_ID
/home/rahulpat/projects/def-khalile2/rahulpat/multiobjbdd/code/multiobj /scratch/rahulpat/knapsack/7/3_20/train/kp_7_3_20_2.dat 1 0 ./output.$SLURM_JOB_ID
/home/rahulpat/projects/def-khalile2/rahulpat/multiobjbdd/code/multiobj /scratch/rahulpat/knapsack/7/3_20/train/kp_7_3_20_3.dat 1 0 ./output.$SLURM_JOB_ID
/home/rahulpat/projects/def-khalile2/rahulpat/multiobjbdd/code/multiobj /scratch/rahulpat/knapsack/7/3_20/train/kp_7_3_20_4.dat 1 0 ./output.$SLURM_JOB_ID
/home/rahulpat/projects/def-khalile2/rahulpat/multiobjbdd/code/multiobj /scratch/rahulpat/knapsack/7/3_20/train/kp_7_3_20_5.dat 1 0 ./output.$SLURM_JOB_ID
/home/rahulpat/projects/def-khalile2/rahulpat/multiobjbdd/code/multiob